In [1]:
import os

In [4]:
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
DIRECTORY = 'DataStore'
path = os.path.join(parent_path, DIRECTORY)

In [5]:
path

'/Users/nnn/GitHub/DataStore'

In [ ]:
import os


def get_path(self, direcotry_name:str = 'DataStore')
# 현재 작업 디렉토리
    current_path = os.getcwd()
    # 상위 폴더 경로
    parent_path = os.path.dirname(current_path)
    # 최종 경로 생성
    path = os.path.join(parent_path, direcotry_name)

    # 폴더 유효성 확인
    if not os.path.exists(path):
        print(f"경로가 존재하지 않습니다: {path}")
    return path
